# Aicua_SR - Symbolic Regression Training on Kaggle GPU

Train PySR models for:
- **Petal Spline V3**: 15 Control Points
- **Bone Rigging V7**: 3 Bones (vertical spine: base → mid → tip)
- **Petal Positioning V1**: MOVE + ROTATE commands with cylindrical coordinates
- **Self-Awareness**: Genesis, Transformation, Composition

## Setup
1. Enable GPU: Settings → Accelerator → GPU P100/T4
2. Clone Aicua_SR repo from GitHub
3. Run training cells

In [ ]:
# Clone Aicua_SR repository
!git clone https://github.com/Aicua/Aicua_SR.git
%cd Aicua_SR

In [ ]:
# Install dependencies
!pip install pysr pandas numpy sympy scikit-learn pyyaml

In [ ]:
# Check GPU availability
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Generate datasets (V3/V7 - latest versions)
!python scripts/generate_petal_spline_v3.py
!python scripts/generate_bone_rigging_v7.py
!python scripts/generate_petal_positioning_v1.py
!python scripts/generate_self_awareness_dataset.py

## Part 1: Petal Spline V3 (15 Control Points)

In [ ]:
# Load petal spline V3 dataset (15 CPs - base narrow like a point, widest at 60-62%)
import pandas as pd
import numpy as np

df = pd.read_csv('data/processed/petal_spline_v3.csv')
print(f"Dataset shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nDataset statistics:")
print(f"  base_size range: {df['base_size'].min():.2f} - {df['base_size'].max():.2f}")
print(f"  Samples: {len(df)}")
df.head()

In [ ]:
# Train models for all 15 control points (V3)
from pysr import PySRRegressor

# Features (v3 naming)
features = ['base_size', 'layer_index', 'petal_index', 'opening_degree']
X = df[features].values

# All targets for 15 CPs (31 total targets)
targets = [
    'cp1_x', 'cp1_y', 'cp2_x', 'cp2_y', 'cp3_x', 'cp3_y', 
    'cp4_x', 'cp4_y', 'cp5_x', 'cp5_y', 'cp6_x', 'cp6_y',
    'cp7_x', 'cp7_y', 'cp8_x', 'cp8_y', 'cp9_x', 'cp9_y',
    'cp10_x', 'cp10_y', 'cp11_x', 'cp11_y', 'cp12_x', 'cp12_y',
    'cp13_x', 'cp13_y', 'cp14_x', 'cp14_y', 'cp15_x', 'cp15_y',
    'extrude_depth'
]

discovered_formulas = {}

for target in targets:
    print(f"\n{'='*60}")
    print(f"Training model for: {target}")
    print(f"{'='*60}")
    
    y = df[target].values
    
    model = PySRRegressor(
        niterations=100,  # Increase for better results (production: 200-500)
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "square"],
        populations=10,
        population_size=30,
        maxsize=20,
        parsimony=0.002,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(X, y, variable_names=features)
    
    formula = str(model.sympy())
    discovered_formulas[target] = formula
    
    print(f"✓ {target} = {formula}")

print(f"\n{'='*60}")
print("All discovered formulas for petal_spline_v3 (15 CPs):")
print(f"{'='*60}")
for target, formula in discovered_formulas.items():
    print(f"{target} = {formula}")

## Part 2: Bone Rigging V7 (3 Bones - Vertical Spine)

In [ ]:
# Load bone_rigging_v7 (3 bones - vertical spine)
print("="*60)
print("BONE RIGGING V7 - 3 Bones (Vertical Spine)")
print("="*60)

bone_df = pd.read_csv('data/processed/bone_rigging_v7.csv')
print(f"Dataset shape: {bone_df.shape}")
print(f"Features: {list(bone_df.columns[:6])}")
print(f"\nBone structure (vertical spine - 3 connected bones):")
print(f"  - bone_base (0% → 45%)   : lower spine")
print(f"  - bone_mid (45% → 78%)   : upper spine (connected)")
print(f"  - bone_tip (78% → 100%)  : tip (connected)")
print(f"\nTargets: 12 position coords + 18 rotation values = 30 total")

# Features for bone rigging v7
bone_features = ['base_size', 'layer_index', 'petal_index', 'opening_degree', 'deformation_type', 'intensity']
bone_X = bone_df[bone_features].values

# Position targets (3 bones × 4 coords = 12)
bone_position_targets = [
    # bone_base
    'bone_base_start_x', 'bone_base_start_y', 'bone_base_end_x', 'bone_base_end_y',
    # bone_mid
    'bone_mid_start_x', 'bone_mid_start_y', 'bone_mid_end_x', 'bone_mid_end_y',
    # bone_tip
    'bone_tip_start_x', 'bone_tip_start_y', 'bone_tip_end_x', 'bone_tip_end_y',
]

# Rotation targets (3 bones × 2 endpoints × 3 axes = 18)
bone_rotation_targets = [
    # bone_base rotations
    'bone_base_head_rx', 'bone_base_head_ry', 'bone_base_head_rz',
    'bone_base_tail_rx', 'bone_base_tail_ry', 'bone_base_tail_rz',
    # bone_mid rotations
    'bone_mid_head_rx', 'bone_mid_head_ry', 'bone_mid_head_rz',
    'bone_mid_tail_rx', 'bone_mid_tail_ry', 'bone_mid_tail_rz',
    # bone_tip rotations
    'bone_tip_head_rx', 'bone_tip_head_ry', 'bone_tip_head_rz',
    'bone_tip_tail_rx', 'bone_tip_tail_ry', 'bone_tip_tail_rz',
]

bone_all_targets = bone_position_targets + bone_rotation_targets

bone_formulas = {}

for target in bone_all_targets:
    print(f"\nTraining: {target}")
    y = bone_df[target].values
    
    model = PySRRegressor(
        niterations=50,  # Increase for production (100-200)
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "sin", "cos"],
        populations=10,
        population_size=30,
        maxsize=15,
        parsimony=0.003,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(bone_X, y, variable_names=bone_features)
    formula = str(model.sympy())
    bone_formulas[target] = formula
    print(f"✓ {target} = {formula}")

print(f"\n{'='*60}")
print("Bone Rigging V7 Formulas (3 bones - vertical spine):")
print(f"{'='*60}")
for target, formula in bone_formulas.items():
    print(f"{target} = {formula}")

## Part 4: Self-Awareness Models

## Part 3: Petal Positioning V1 (MOVE + ROTATE Commands)

## Part 3: Self-Awareness Models

In [ ]:
# Load Self-Awareness Genesis dataset
print("="*60)
print("SELF-AWARENESS: GENESIS")
print("="*60)

genesis_df = pd.read_csv('data/generated/sa_genesis_dataset.csv')
print(f"Dataset shape: {genesis_df.shape}")
print(f"Columns: {list(genesis_df.columns)}")

# Genesis features and targets
genesis_features = ['layer', 'width', 'height', 'aspect_ratio', 'detail_code', 'opening_degree']
genesis_targets = ['cp_count', 'self_understanding', 'structural_confidence']

genesis_X = genesis_df[genesis_features].values

genesis_formulas = {}

for target in genesis_targets:
    print(f"\nTraining: {target}")
    y = genesis_df[target].values
    
    model = PySRRegressor(
        niterations=50,
        binary_operators=["+", "-", "*", "/"],
        unary_operators=["sqrt", "exp", "log"],
        populations=15,
        population_size=33,
        maxsize=20,
        parsimony=0.0032,
        model_selection="best",
        verbosity=0,
    )
    
    model.fit(genesis_X, y, variable_names=genesis_features)
    formula = str(model.sympy())
    r2 = model.score(genesis_X, y)
    genesis_formulas[target] = {'formula': formula, 'r2': r2}
    print(f"✓ {target} = {formula}")
    print(f"  R² = {r2:.4f}")

print("\nGenesis Formulas:")
for target, data in genesis_formulas.items():
    print(f"  {target}: {data['formula']} (R²={data['r2']:.4f})")

## Part 5: Save All Formulas

In [ ]:
# Save all discovered formulas to JSON
import json
import os

os.makedirs('data/generated', exist_ok=True)

all_formulas = {
    'petal_spline_v3': discovered_formulas,
    'bone_rigging_v7': bone_formulas,
    'petal_positioning_v1': pos_formulas,
    'self_awareness': {
        'genesis': genesis_formulas,
        'transformation': transform_formulas,
        'composition': composition_formulas,
    }
}

with open('data/generated/sr_discovered_formulas_v7.json', 'w') as f:
    json.dump(all_formulas, f, indent=2, default=str)

print("✓ Saved all formulas to data/generated/sr_discovered_formulas_v7.json")
print(f"\nTotal formulas discovered:")
print(f"  - Petal Spline V3 (15 CPs): {len(discovered_formulas)} targets")
print(f"  - Bone Rigging V7 (5 bones): {len(bone_formulas)} targets")
print(f"  - Petal Positioning V1: {len(pos_formulas)} targets")
print(f"  - Genesis: {len(genesis_formulas)} targets")
print(f"  - Transformation: {len(transform_formulas)} targets")
print(f"  - Composition: {len(composition_formulas)} targets")

# Generate Python code from discovered formulas
def formula_to_python(formula_str, target_name, features):
    """Convert SymPy formula to Python function."""
    code = formula_str
    
    # Replace math functions
    code = code.replace('sqrt', 'math.sqrt')
    code = code.replace('exp', 'math.exp')
    code = code.replace('log', 'math.log')
    code = code.replace('sin', 'math.sin')
    code = code.replace('cos', 'math.cos')
    
    params = ', '.join(features)
    
    func_code = f"""
def compute_{target_name}({params}):
    \"\"\"SR-discovered formula for {target_name}.\"\"\"
    try:
        return {code}
    except (ValueError, ZeroDivisionError):
        return 0.0
"""
    return func_code

# Generate Python module
module_code = '''#!/usr/bin/env python3
"""Auto-generated SR formulas for Aicua_SR V7.

Generated by Kaggle GPU training.
- Petal Spline V3: 15 Control Points
- Bone Rigging V7: 3 Bones (Vertical Spine)
- Petal Positioning V1: MOVE + ROTATE commands
- Self-Awareness: Genesis, Transformation, Composition
"""
import math

'''

# Add petal spline formulas
module_code += "# " + "="*60 + "\n"
module_code += "# PETAL SPLINE V3 (15 Control Points)\n"
module_code += "# " + "="*60 + "\n\n"

petal_features = ['base_size', 'layer_index', 'petal_index', 'opening_degree']
for target, formula in discovered_formulas.items():
    module_code += formula_to_python(formula, f'petal_{target}', petal_features)

# Add bone rigging formulas
module_code += "\n# " + "="*60 + "\n"
module_code += "# BONE RIGGING V7 (3 Bones - Vertical Spine)\n"
module_code += "# " + "="*60 + "\n\n"

for target, formula in bone_formulas.items():
    module_code += formula_to_python(formula, f'bone_{target}', bone_features)

# Add petal positioning formulas
module_code += "\n# " + "="*60 + "\n"
module_code += "# PETAL POSITIONING V1 (MOVE + ROTATE)\n"
module_code += "# " + "="*60 + "\n\n"

for target, data in pos_formulas.items():
    module_code += formula_to_python(data['formula'], f'pos_{target}', pos_features)

# Save to file
with open('data/generated/aicua_sr_formulas_v7.py', 'w') as f:
    f.write(module_code)

print("✓ Generated Python module: data/generated/aicua_sr_formulas_v7.py")

## Summary

### Trained Models:

1. **Petal Spline V3** - 31 targets (15 CPs × 2 coords + extrude_depth)
   - Base NARROW like a point (5% height)
   - WIDEST at 60-62% height
   - Rounded tip at 100%

2. **Bone Rigging V7** - 30 targets (3 bones: 12 positions + 18 rotations)
   - Vertical spine structure (3 connected bones):
     - **bone_base** (0→45%): lower spine
     - **bone_mid** (45→78%): upper spine (connected)
     - **bone_tip** (78→100%): tip (connected)
   - Position targets: 3 bones × 4 coords = 12
   - Rotation targets: 3 bones × 2 endpoints × 3 axes = 18

3. **Petal Positioning V1** - 6 targets (MOVE + ROTATE commands)
   - Cylindrical coordinates → Cartesian conversion
   - Features: layer_radius, num_petals, petal_index, opening_degree, base_tilt_angle, z_variation
   - Position targets: pos_x, pos_y, pos_z
   - Rotation targets: rotate_x, rotate_y, rotate_z
   - Perfect symmetry for circular petal arrangements

4. **Self-Awareness**
   - Genesis: cp_count, self_understanding, structural_confidence
   - Transformation: morph_confidence, avg_risk_level, structural_stability
   - Composition: harmony_score, cooperation_confidence

### Total Formulas: 31 + 30 + 6 + 8 = **75 formulas**

### Next Steps:

1. Increase iterations for production (200-500 for petal/bone, 100-200 for positioning/SA)
2. Download `sr_discovered_formulas_v7.json` and `aicua_sr_formulas_v7.py`
3. Integrate formulas into Aicua_SR CLI generators
4. Test generated `.cli` files with MOVE+ROTATE commands

In [ ]:
# Generate Python code from discovered formulas
def formula_to_python(formula_str, target_name, features):
    """Convert SymPy formula to Python function."""
    code = formula_str
    
    # Replace math functions
    code = code.replace('sqrt', 'math.sqrt')
    code = code.replace('exp', 'math.exp')
    code = code.replace('log', 'math.log')
    code = code.replace('sin', 'math.sin')
    code = code.replace('cos', 'math.cos')
    
    params = ', '.join(features)
    
    func_code = f"""
def compute_{target_name}({params}):
    \"\"\"SR-discovered formula for {target_name}.\"\"\"
    try:
        return {code}
    except (ValueError, ZeroDivisionError):
        return 0.0
"""
    return func_code

# Generate Python module
module_code = '''#!/usr/bin/env python3
"""Auto-generated SR formulas for Aicua_SR V6.

Generated by Kaggle GPU training.
- Petal Spline V3: 15 Control Points
- Bone Rigging V6: 5 Independent Bones (no parent-child)
- Self-Awareness: Genesis, Transformation, Composition
"""
import math

'''

# Add petal spline formulas
module_code += "# " + "="*60 + "\n"
module_code += "# PETAL SPLINE V3 (15 Control Points)\n"
module_code += "# " + "="*60 + "\n\n"

petal_features = ['base_size', 'layer_index', 'petal_index', 'opening_degree']
for target, formula in discovered_formulas.items():
    module_code += formula_to_python(formula, f'petal_{target}', petal_features)

# Add bone rigging formulas
module_code += "\n# " + "="*60 + "\n"
module_code += "# BONE RIGGING V6 (5 Independent Bones)\n"
module_code += "# " + "="*60 + "\n\n"

for target, formula in bone_formulas.items():
    module_code += formula_to_python(formula, f'bone_{target}', bone_features)

# Save to file
with open('data/generated/aicua_sr_formulas_v6.py', 'w') as f:
    f.write(module_code)

print("✓ Generated Python module: data/generated/aicua_sr_formulas_v6.py")

## Summary

### Trained Models:

1. **Petal Spline V3** - 17 targets (8 CPs × 2 coords + extrude_depth)
   - Heart-shaped tip with notch (CP4, CP5, CP6)
   - Undulating edges (CP2, CP3, CP7, CP8)

2. **Bone Rigging V5** - 28 targets (7 bones × 4 coords)
   - Central spine: root → middle → tip
   - Left ribs: lower, upper
   - Right ribs: lower, upper

3. **Self-Awareness**
   - Genesis: cp_count, self_understanding, structural_confidence
   - Transformation: morph_confidence, avg_risk_level, structural_stability
   - Composition: harmony_score, cooperation_confidence

### Next Steps:

1. Increase iterations for production (100-500)
2. Download `sr_discovered_formulas_v3.json`
3. Integrate formulas into Aicua_SR CLI